# Load libraries and functions

In [45]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [46]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

## Main Code

In [47]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# occupation = 'dredgeOperators'
# occupation = 'gradersAndSorters'
# occupation = 'reinforcingIron'
# occupation = 'insuranceAppraisers'
# occupation = 'floorSanders'
# occupation = 'dataEntryKeyer'
# occupation = 'athletesAndSportsCompetitors'
# # occupation = 'shampooers'

GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [48]:
# Load the data
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
tasks

['Collect payment for transportation and accommodations from customer.',
 'Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.',
 "Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tour's costs.",
 'Book transportation and hotel reservations, using computer or telephone.',
 'Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.',
 'Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.',
 'Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.',
 'Record and maintain information on clients, vendors, and travel packages.']

<br>

<br>

# 5) GPT Triangles/Conditioning Method

#### Approach: Use either First-Last Task method or Partitioned Tasks Method in creating original GPT DAG. Next use the "Triangles" or "Conditioning" method for narrowing down set of edges.

In [49]:
version = ''

In [50]:
input_version = 'naive'
input_version = 'firstLastTask'
input_version = 'partitioned'

### Set input-output file names for different versions of the run

In [51]:
if input_version == 'naive':
    input_filename = f'{occupation_folder}/{version}{occupation}_N_GPT_DAG_df.csv'
    output_filename = f'{occupation_folder}/{version}{occupation}_CN_GPT_DAG_df.csv'

if input_version == 'firstLastTask':
    input_filename = f'{occupation_folder}/{version}{occupation}_FLT_GPT_DAG_df.csv'
    output_filename = f'{occupation_folder}/{version}{occupation}_CFLT_GPT_DAG_df.csv'

if input_version == 'partitioned':
    input_filename = f'{occupation_folder}/{version}{occupation}_P_GPT_DAG_df.csv'
    output_filename = f'{occupation_folder}/{version}{occupation}_CP_GPT_DAG_df.csv'

### Set up questions and options

In [52]:
triangle_check_question_text = dedent("""\
                                        Consider {{ occupation }} as an occupation. 
                                        And consider these three tasks: 
                                        A) {{ task_A }} 
                                        B) {{ task_B }}
                                        C) {{ task_C }} 
                                        Imagine there are two workers, one is working on task A and the other is working on task B.
                                        Suppose a third worker wants to start working on task C.
                                        Does the third worker need to wait for the workers working on tasks A and B to finish before starting on task C?
                                        Or can the third worker start working on task C right after worker B finishes without needing output of worker A?
                                        Avoid using words like "task A", "task B", or "task C" in the answer.
                                        Explain the reasoning behind your answer in a couple of sentences.
                                        """)
triangle_check_question_options = {'keep AC': "Third worker can only start C after both the first and second workers have finished tasks A and B",
                                    'drop AC': "Third worker can start C after the second worker finishes B without having to wait for the first worker to finish A",
                                    'sanity check': "These are not part of the same task sequence"
                                    }

AC_DC_question_question_text = dedent("""\
                                        Consider {{ occupation }} as an occupation.
                                        And consider these tasks:
                                        A) {{ task_A }}
                                        B) {{ task_B }}
                                        C) {{ task_C }}
                                        D) {{ task_D }}         
                                        As part of the steps leading up to completion of this job {{ task_C }} must be done.
                                        We know that tasks A, B, and D are inputs to task C.Moreover, task D is an input to task A while task A is an input to task B.
                                        Imagine there are three workers, one is working on task A, one is working on task B, and the other is working on task D.
                                        Suppose a fourth worker wants to start working on task C.
                                        Does the fourth worker need to wait for the workers working on tasks A, B, and D to finish before starting on task C?
                                        Or can the third worker start working on task C after workers B and D finish without needing output of worker A?
                                        How about after workers B and A finish without needing output of worker D?
                                        Avoid using words like "task A", "task B", or "task C" in the answer.
                                        Explain the reasoning behind your answer in a couple of sentences.
                                        """)
AC_DC_question_options = {'drop AC, drop DC': "Given that worker B is finished, output of neither worker A nor worker D is needed for worker C",
                            'keep AC, drop DC': "Given that worker B is finished, output of worker A is needed for worker C but output of worker D is not",
                            'drop AC, keep DC': "Given that worker B is finished, output of worker D is needed for worker C but output of worker A is not",
                            'keep AC, keep DC': "Given that worker B is finished, output of both workers A and D are needed for worker C",
                            'sanity check': "These are not part of the same task sequence"
                            }

In [53]:
triangle_check_question_options_list = list(triangle_check_question_options.values())
AC_DC_question_options_list = list(AC_DC_question_options.values())

In [54]:
def triangle_check(occupation, tasks, triangles_list, question_text, question_options_list):
    triangles = np.array(triangles_list)
    task_A_list = triangles[:, 0]
    task_B_list = triangles[:, 1]
    task_C_list = triangles[:, 2]
    scenarios = [Scenario({"occupation": occupation, "task_A": tasks[task_A], "task_B": tasks[task_B], "task_C": tasks[task_C]}) 
        for task_A, task_B, task_C in zip(task_A_list, task_B_list, task_C_list)]

    q = QuestionMultipleChoice(
        question_name = "ordering",
        question_text = question_text,
        question_options = question_options_list
    )
    results = q.by(m4).by(scenarios).run(progress_bar = True)
    return results



q_AC_DC = QuestionMultipleChoice(
    question_name = "AC_DC",
    question_text = AC_DC_question_question_text,
    question_options = AC_DC_question_options_list
    )

<br> 

### Step 1:

#### Find all "triangles", defined as cases with:
##### A --> B --> C
##### A --> C

In [55]:
# Read output of one step GPT DAG
GPT_AM_df = pd.read_csv(input_filename)

# Remove "Target" node for conditioning analysis. Will add later
GPT_AM_df_targetNodes = GPT_AM_df[GPT_AM_df.target == '"Target"']
GPT_AM_df = GPT_AM_df[GPT_AM_df.target != '"Target"']

In [56]:
# Convert GPT AM data frame to adjacency matrix
GPT_AM = pd.DataFrame(0, index=tasks, columns=tasks)
for index, row in GPT_AM_df.iterrows():
    GPT_AM.at[row['source'], row['target']] = 1

In [57]:
def find_triangles(matrix):
    # Ensure matrix is a numpy array
    if not isinstance(matrix, np.ndarray):
        matrix = matrix.to_numpy()
    
    # get length of matrix
    n = matrix.shape[0]

    # create list containing integers from 0 to n-1 for indexing
    numbers = list(range(n))

    # Find triangles
    triangles = []
    for x, y, z in itertools.permutations(numbers, 3):
        # get indices of destination nodes for outgoing edges of x
        out_edges_destination_x = np.where(matrix[x] == 1)[0]
        out_edges_destination_x = list(out_edges_destination_x)

        # check if x has outgoing edge to both y and z
        # if yes, check if y has outgoing edge to z
        if y in out_edges_destination_x and z in out_edges_destination_x:
            out_edges_destination_y = np.where(matrix[y] == 1)[0]
            out_edges_destination_y = list(out_edges_destination_y)
            
            # check if y has outgoing edge to z
            # if yes, we have a triangle
            if z in out_edges_destination_y:
                triangles.append([x, y, z])
    
    return triangles

# Find triangles
GPT_AM_triangles_list = find_triangles(GPT_AM)
print(f'Examples of triangles: {GPT_AM_triangles_list[:5]}')
print(f'Count of triangles: {len(GPT_AM_triangles_list)}')

Examples of triangles: [[0, 3, 6], [0, 3, 7], [0, 6, 7], [1, 0, 3], [1, 0, 6]]
Count of triangles: 49


In [58]:
# If there are no triangles found, export input as output (as conditioning method doesn't change anything)
if len(GPT_AM_triangles_list) == 0:
    # Add back "Target" node and save original input as output
    GPT_AM_df = pd.concat([GPT_AM_df, GPT_AM_df_targetNodes], ignore_index=True)
    GPT_AM_df.to_csv(output_filename, index=False)

### Step 2: 
#### Ask GPT whether conditional on having B --> C we need A --> C

In [59]:
results = triangle_check(GPT_input_occupation, tasks, GPT_AM_triangles_list, triangle_check_question_text, triangle_check_question_options_list)
#results.select("task_A", "task_B", "task_C", "ordering", "comment.ordering_comment").print()
GPT_trianglesCheck_output = results.select("task_A", "task_B", "task_C", "ordering", "comment.ordering_comment").to_pandas()
GPT_trianglesCheck_output = GPT_trianglesCheck_output.sort_values(by=['scenario.task_A', 'scenario.task_C', 'scenario.task_B']).reset_index(drop=True)

Output()

### In cases where A --> C is shared among multiple triangles, only delete when all triangles say delete

In [60]:
# Step 1: Find the count of triangles for each A --> C pair
GPT_trianglesCheck_output['AC_pair_triangles_count'] = GPT_trianglesCheck_output.groupby(['scenario.task_A', 'scenario.task_C'])['scenario.task_A'].transform('count')


# Step 2: Find if all triangles say delete
aux_df = GPT_trianglesCheck_output.groupby(['scenario.task_A', 'scenario.task_C'])['answer.ordering'].apply(lambda x: (x == triangle_check_question_options['drop AC']).mean()*100).reset_index()
aux_df.columns = ['scenario.task_A', 'scenario.task_C', 'fraction_triangles_say_delete']
aux_df = aux_df[aux_df['fraction_triangles_say_delete'] == 100]

# Merge aux_df with the original DataFrame to keep 'comment.ordering_comment'
edges_to_remove = pd.merge(aux_df, GPT_trianglesCheck_output, on=['scenario.task_A', 'scenario.task_C'])
edges_to_remove['comment.ordering_comment'] = (
    edges_to_remove['comment.ordering_comment'] 
    + '\n\n(Source):\n' + edges_to_remove['scenario.task_A'] 
    + '\n(Conditioned on):\n' + edges_to_remove['scenario.task_B'] 
    + '\n(Target):\n' + edges_to_remove['scenario.task_C']
)
edges_to_remove = edges_to_remove[['scenario.task_A', 'scenario.task_C', 'comment.ordering_comment']]

# Step 3: Delete the rows where all triangles say delete
modified_GPT_trianglesCheck = pd.merge(GPT_trianglesCheck_output, edges_to_remove[['scenario.task_A', 'scenario.task_C']], 
                                       how='left', 
                                       on=['scenario.task_A', 'scenario.task_C'], 
                                       indicator=True)
modified_GPT_trianglesCheck = modified_GPT_trianglesCheck[modified_GPT_trianglesCheck['_merge'] == 'left_only'].drop(columns=['_merge', 'AC_pair_triangles_count'])
modified_GPT_trianglesCheck = modified_GPT_trianglesCheck.reset_index(drop=True)
edges_to_remove

,scenario.task_A,scenario.task_C,comment.ordering_comment
0,"Book transportation and hotel reservations, us...",Print or request transportation carrier ticket...,The third worker can start printing or request...
1,Collect payment for transportation and accommo...,Print or request transportation carrier ticket...,The third worker can start printing or request...
2,"Compute cost of travel and accommodations, usi...","Book transportation and hotel reservations, us...",The third worker can start booking transportat...
3,"Compute cost of travel and accommodations, usi...",Collect payment for transportation and accommo...,The third worker can start collecting payment ...
4,"Compute cost of travel and accommodations, usi...","Plan, describe, arrange, and sell itinerary to...",The third worker can start planning and sellin...
5,"Plan, describe, arrange, and sell itinerary to...",Print or request transportation carrier ticket...,The third worker can start printing or request...
6,Provide customer with brochures and publicatio...,"Book transportation and hotel reservations, us...",The third worker can begin booking transportat...
7,Provide customer with brochures and publicatio...,Collect payment for transportation and accommo...,The third worker can start collecting payment ...
8,Provide customer with brochures and publicatio...,"Plan, describe, arrange, and sell itinerary to...",The third worker can start planning and arrang...
9,Provide customer with brochures and publicatio...,Print or request transportation carrier ticket...,The third worker can start printing or request...


#### Create a variable saying how many times each node appears as which node in a triangle
##### Purpose: find quanrangles

In [61]:
# Initialize an empty DataFrame with unique values as columns and original columns as rows
aux_df = pd.DataFrame(0, index=['scenario.task_A', 'scenario.task_B', 'scenario.task_C'], columns=tasks)

# Fill the new DataFrame with counts
for col in modified_GPT_trianglesCheck[['scenario.task_A', 'scenario.task_B', 'scenario.task_C']].columns:
    value_counts = modified_GPT_trianglesCheck[col].value_counts()
    aux_df.loc[col, value_counts.index] = value_counts.values
aux_df = aux_df.T

# Keep tasks which are sometimes node A of a triangle and sometimes node B of a triangle
#aux_df = aux_df[(aux_df > 0).all(axis=1)]
print('Nodes stats as nodes A, B, C of a triangle:')
aux_df

# get list of pivotal tasks
#pivotal_tasks = aux_df.index.tolist()

Nodes stats as nodes A, B, C of a triangle:


,scenario.task_A,scenario.task_B,scenario.task_C
Collect payment for transportation and accommodations from customer.,2,7,3
"Converse with customer to determine destination, mode of transportation, travel dates, financial considerations, and accommodations required.",20,0,0
"Compute cost of travel and accommodations, using calculator, computer, carrier tariff books, and hotel rate books, or quote package tour's costs.",7,5,0
"Book transportation and hotel reservations, using computer or telephone.",2,7,3
"Plan, describe, arrange, and sell itinerary tour packages and promotional travel incentives offered by various travel carriers.",2,7,3
"Provide customer with brochures and publications containing travel information, such as local customs, points of interest, or foreign country regulations.",4,5,0
"Print or request transportation carrier tickets, using computer printer system or system link to travel carrier.",0,6,8
"Record and maintain information on clients, vendors, and travel packages.",0,0,20


### Step 3: 
##### In cases where 
A --> B --> C and D --> A --> C 
##### the situation is different from when 
A --> B --> C and A --> D --> C
##### In such cases, edges A --> C and D --> C must be considered simultaneously as triangles are not totally "independent". 
#### So we look for "quadrangles"

In [62]:
# Iterate over the list of tuples and subset the DataFrame
quadrangles_tasks = []
for A, B, C, D in itertools.permutations(tasks, 4):
    # Initialize an empty list to collect the indices of desired rows
    quadrangle_indices = []

    # Find rows where triangle nodes are A, B, C
    condition1 = (modified_GPT_trianglesCheck['scenario.task_A'] == A) & (modified_GPT_trianglesCheck['scenario.task_B'] == B) & (modified_GPT_trianglesCheck['scenario.task_C'] == C)
    rows1 = modified_GPT_trianglesCheck[condition1]
    
    # Find rows where triangle nodes are D, A, C
    condition2 = (modified_GPT_trianglesCheck['scenario.task_A'] == D) & (modified_GPT_trianglesCheck['scenario.task_B'] == A) & (modified_GPT_trianglesCheck['scenario.task_C'] == C)
    rows2 = modified_GPT_trianglesCheck[condition2]
    
    # If both conditions are met, add the indices to the list
    if not rows1.empty and not rows2.empty:
        quadrangles_tasks.append((A, B, C, D))    
print(f'Number of quadrilaterals: {len(quadrangles_tasks)}')

Number of quadrilaterals: 35


In [63]:
quadrangles_df = pd.DataFrame()
if len(quadrangles_tasks) > 0:
    scenarios = [Scenario({"occupation": GPT_input_occupation, "tasks": tasks,
                "task_A": A, "task_B": B, "task_C": C, "task_D": D})
                for A, B, C, D in quadrangles_tasks]
    results_AC_DC = q_AC_DC.by(m4).by(scenarios).run()
    #results_AC_DC.select(['answer.AC_DC', 'scenario.task_A', 'scenario.task_B', 'scenario.task_C', 'scenario.task_D', 'comment.AC_DC_comment']).print()
    quadrangles_df = results_AC_DC.select(['answer.AC_DC', 'scenario.task_A', 'scenario.task_B', 'scenario.task_C', 'scenario.task_D', 'comment.AC_DC_comment']).to_pandas()

    # decide whether to keep or drop AC and DC
    quadrangles_df['keep_AC'] = quadrangles_df['answer.AC_DC'].apply(lambda x: x in [AC_DC_question_options['keep AC, keep DC'], AC_DC_question_options['keep AC, drop DC']])
    quadrangles_df['keep_DC'] = quadrangles_df['answer.AC_DC'].apply(lambda x: x in [AC_DC_question_options['keep AC, keep DC'], AC_DC_question_options['drop AC, keep DC']])

    # Add node info to comments
    quadrangles_df['comment.AC_DC_comment'] = (
        quadrangles_df['comment.AC_DC_comment'] 
        + '\n\n(Source):\n' + quadrangles_df['scenario.task_A'] 
        + '\n(Conditioned on):\n' + quadrangles_df['scenario.task_B'] 
        + '\n(Target):\n' + quadrangles_df['scenario.task_C']
        + '\n(Other task -- Task D):\n' + quadrangles_df['scenario.task_D']
    )
quadrangles_df.head()

,answer.AC_DC,comment.AC_DC_comment,scenario.task_A,scenario.task_B,scenario.task_C,scenario.task_D,keep_AC,keep_DC
0,"Given that worker B is finished, output of bot...",The fourth worker needs to wait for the comple...,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...","Record and maintain information on clients, ve...",Converse with customer to determine destinatio...,True,True
1,"Given that worker B is finished, output of bot...",The fourth worker needs the outputs from all t...,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...","Record and maintain information on clients, ve...","Compute cost of travel and accommodations, usi...",True,True
2,"Given that worker B is finished, output of bot...",Since recording and maintaining information on...,Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...","Record and maintain information on clients, ve...","Plan, describe, arrange, and sell itinerary to...",True,True
3,"Given that worker B is finished, output of bot...","To record and maintain information on clients,...",Collect payment for transportation and accommo...,"Book transportation and hotel reservations, us...","Record and maintain information on clients, ve...",Provide customer with brochures and publicatio...,True,True
4,"Given that worker B is finished, output of bot...",Since the task of recording and maintaining in...,Collect payment for transportation and accommo...,Print or request transportation carrier ticket...,"Record and maintain information on clients, ve...",Converse with customer to determine destinatio...,True,True


#### Drop extra AC and DC edges

In [64]:
ACDC_edges_to_remove = pd.DataFrame()
if len(quadrangles_tasks) > 0:
    # Step 1: Get list of unique edges found in all quadrangles
    pairs_AC = list(zip(quadrangles_df["scenario.task_A"], quadrangles_df["scenario.task_C"]))
    pairs_AC = [(task_A, task_C, 'AC') for (task_A, task_C) in pairs_AC]
    pairs_DC = list(zip(quadrangles_df["scenario.task_D"], quadrangles_df["scenario.task_C"]))
    pairs_DC = [(task_D, task_C, 'DC') for (task_D, task_C) in pairs_DC]
    all_pairs = pairs_AC + pairs_DC


    # Step 2: Get list of edges to keep
    aux_df = quadrangles_df[quadrangles_df['keep_AC']==True]
    pairs_AC_toKeep = list(zip(aux_df["scenario.task_A"], aux_df["scenario.task_C"]))
    pairs_AC_toKeep = [(task_A, task_C, 'AC') for (task_A, task_C) in pairs_AC_toKeep]

    aux_df = quadrangles_df[quadrangles_df['keep_DC']==True]
    pairs_DC_toKeep = list(zip(aux_df["scenario.task_D"], aux_df["scenario.task_C"]))
    pairs_DC_toKeep = [(task_D, task_C, 'DC') for (task_D, task_C) in pairs_DC_toKeep]

    pairs_toKeep = pairs_AC_toKeep + pairs_DC_toKeep


    # Step 3: Get list of edges to drop
    ACDC_edges_toDrop_list = [item for item in all_pairs if item not in pairs_toKeep]
    ACDC_edges_to_remove = pd.DataFrame(ACDC_edges_toDrop_list, columns=["scenario.task_A", "scenario.task_C", 'ID'])


    # Step 4: Match comments
    AC_indices = ACDC_edges_to_remove[ACDC_edges_to_remove['ID'] == 'AC'].index
    DC_indices = ACDC_edges_to_remove[ACDC_edges_to_remove['ID'] == 'DC'].index

    # Split ACDC_edges_to_remove into two DataFrames based on AC or DC edges
    aux_df_AC = ACDC_edges_to_remove.loc[AC_indices]#[['scenario.task_A', 'scenario.task_C']]
    aux_df_DC = ACDC_edges_to_remove.loc[DC_indices]#[['scenario.task_A', 'scenario.task_C']]

    # Merge comments depending on AC or DC edge
    merged_AC = pd.merge(aux_df_AC, quadrangles_df[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']], 
                        on=['scenario.task_A', 'scenario.task_C'], 
                        how='left')
    merged_AC = merged_AC[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']]
    merged_DC = pd.merge(aux_df_DC, quadrangles_df[['scenario.task_D', 'scenario.task_C', 'comment.AC_DC_comment']], 
                        left_on=['scenario.task_A', 'scenario.task_C'], 
                        right_on=['scenario.task_D', 'scenario.task_C'], 
                        how='left')
    merged_DC = merged_DC[['scenario.task_A', 'scenario.task_C', 'comment.AC_DC_comment']]

    # Concatenate merged DataFrames
    ACDC_edges_to_remove = pd.concat([merged_AC, merged_DC]).sort_index()
    ACDC_edges_to_remove = ACDC_edges_to_remove.drop_duplicates(subset=['scenario.task_A', 'scenario.task_C']).reset_index(drop=True)
    ACDC_edges_to_remove.columns = ['scenario.task_A', 'scenario.task_C', 'comment.ordering_comment'] # for consistency with previous edges_to_remove data frame


# Create a DataFrame of edges to be dropped from this analysis and earlier analyses
print(f'Number of AC-DC edges to remove: {len(ACDC_edges_to_remove)}')
print(f'Number of AC edges to remove: {len(edges_to_remove)}')
edges_to_remove = pd.concat([edges_to_remove, ACDC_edges_to_remove], ignore_index=True)
print(f'Total number of edges to remove: {len(edges_to_remove)}')

Number of AC-DC edges to remove: 0
Number of AC edges to remove: 12
Total number of edges to remove: 12


In [65]:
# Label edges to remove
def adjust_graph_label_for_removed_edges(edges_to_remove, main_df):
    output = main_df.copy()
    for _, row in edges_to_remove.iterrows():
        task_A, task_C, comment = row['scenario.task_A'], row['scenario.task_C'], row['comment.ordering_comment']
        match = (main_df['source'] == task_A) & (main_df['target'] == task_C)
        if match.any():
            index_to_update = main_df.index[match][0]  # Get the index where the pair matches
            output.at[index_to_update, 'comment'] = comment + 'TriangleRemovedFlag'
    return output
modified_GPT_AM_df = adjust_graph_label_for_removed_edges(edges_to_remove, GPT_AM_df)

In [66]:
# Add back "Target" node edges
modified_GPT_AM_df = pd.concat([modified_GPT_AM_df, GPT_AM_df_targetNodes], ignore_index=True)

# Save output
modified_GPT_AM_df.to_csv(output_filename, index=False)

<br>

<br>

<br>

<br>

<br>